In [3]:
from finrl.train import train
from finrl.test import test
from finrl.config_tickers import DOW_30_TICKER
from finrl.config import INDICATORS
from finrl.meta.env_stock_trading.env_stocktrading_np import StockTradingEnv
from finrl.meta.env_stock_trading.env_stock_papertrading import AlpacaPaperTrading
from finrl.meta.data_processor import DataProcessor
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

import numpy as np
import pandas as pd

# install ipywidgets to fix the jupyter notebook warning
# D:\Anaconda3\envs\finrl\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
#   from .autonotebook import tqdm as notebook_tqdm
# pip install ipywidgets

D:\Anaconda3\envs\finrl\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [4]:
ticker_list = DOW_30_TICKER
action_dim = len(DOW_30_TICKER)
candle_time_interval = '15Min'

In [5]:
print(ticker_list)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [6]:
print(INDICATORS)

['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']


In [7]:
# amount + (turbulence, turbulence_bool) + (price, shares, cd (holding time)) * stock_dim + tech_dim
state_dim = 1 + 2 + 3 * action_dim + len(INDICATORS) * action_dim
state_dim

333

In [34]:
API_KEY = "PKZOWRCNANQ79J7CV314"
API_SECRET = "7xYTpMGdeffzOsDDsrnl4UryzvJClIpsvZeTIPYf"
API_BASE_URL = 'https://paper-api.alpaca.markets'
data_url = 'wss://data.alpaca.markets'
env = StockTradingEnv

## Alpaca API

In [51]:
# import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame, TimeFrameUnit

In [52]:
try:
    api = REST(API_KEY, API_SECRET, API_BASE_URL, "v2")
except BaseException:
    raise ValueError("Wrong Account Info!")

In [58]:
TimeFrame(30, TimeFrameUnit.Minute)
"30Min"

'30Min'

In [56]:
api.get_bars("AAPL", TimeFrame(30, TimeFrameUnit.Minute), "2021-06-08", "2021-06-08", adjustment='raw').df

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2021-06-08 07:30:00+00:00,126.1000,126.1600,125.9600,126.0600,20951,304,126.049447
2021-06-08 08:15:00+00:00,126.0500,126.3000,126.0500,126.3000,21181,349,126.231904
2021-06-08 09:00:00+00:00,126.2700,126.3200,126.2200,126.2800,15955,308,126.284120
2021-06-08 09:45:00+00:00,126.2900,126.4000,125.9000,125.9000,30179,582,126.196877
2021-06-08 10:30:00+00:00,125.9000,126.7500,125.8400,126.7500,105380,1376,126.530863
2021-06-08 11:15:00+00:00,126.7300,126.8500,126.5600,126.8300,129721,1760,126.738041
2021-06-08 12:00:00+00:00,126.4101,126.9500,126.3999,126.8300,418107,3615,126.771889
2021-06-08 12:45:00+00:00,126.8500,126.9400,126.6000,126.6200,428614,5526,126.802825
2021-06-08 13:30:00+00:00,126.6200,128.4600,126.4485,127.4150,23064810,171259,127.425804


In [37]:
tic="AXP"
time_interval=candle_time_interval
NY = "America/New_York"
start_date = pd.Timestamp("2011-07-01", tz=NY)
end_date = pd.Timestamp("2011-07-02", tz=NY)
start_time=(start_date + pd.Timedelta("09:30:00")).isoformat()
end_time=(start_date + pd.Timedelta("15:59:00")).isoformat()

In [38]:
start_time

'2011-07-01T09:30:00-04:00'

In [39]:
end_time

'2011-07-01T15:59:00-04:00'

In [40]:
data_df = pd.DataFrame()

In [41]:
barset = api.get_bars(tic, time_interval, start=start_time, end=end_time, limit=500).df

In [42]:
barset

""


In [24]:
barset["tic"] = tic
barset = barset.reset_index()
data_df = pd.concat([data_df, barset])

ValueError: cannot insert level_0, already exists

In [7]:
DP = DataProcessor(data_source = 'alpaca',
                  API_KEY = API_KEY, 
                  API_SECRET = API_SECRET, 
                  API_BASE_URL = API_BASE_URL
                  )

Alpaca successfully connected


In [8]:
data = DP.download_data(start_date = '2021-10-01', 
                        end_date = '2021-10-02',
                        ticker_list = ticker_list, 
                        time_interval= candle_time_interval,
                        dataset_path='../../../epocha/dataset/stock')

TTTTT
Data before 2021-10-01T15:59:00-04:00 is successfully fetched
Data before 2021-10-02T15:59:00-04:00 is successfully fetched
